In [118]:
import numpy as np
import pandas as pd
import MeCab
import sys
from collections import Counter
import re
import urllib.request, urllib.error

In [119]:
tagger = MeCab.Tagger("-Owakati")
tagger.parse("")

'\n'

In [120]:
lunch_file = ["../pretest_sushitsuu_lunch.csv",
              "../pretest_sushiichi_lunch.csv",
              "../pretest_sawada_lunch.csv",
              "../pretest_mitani_lunch.csv",
              "../pretest_matsukan_lunch.csv",
              "../pretest_kiyoda_lunch.csv",
              "../pretest_imamura_lunch.csv",
              "../pretest_hatsune_lunch.csv"
              ]

In [121]:
dinner_file = ["../pretest_sushitsuu_dinner.csv",
              "../pretest_sushiichi_dinner.csv",
              "../pretest_sawada_dinner.csv",
              "../pretest_mitani_dinner.csv",
              "../pretest_matsukan_dinner.csv",
              "../pretest_kiyoda_dinner.csv",
              "../pretest_imamura_dinner.csv",
              "../pretest_hatsune_dinner.csv",
              "../pretest_aozora_dinner.csv"
              ]

In [122]:
store_name = ["sushitsuu",
             "sushiichi",
             "sawada",
             "mitani",
             "matsukan",
             "kiyoda",
             "imamura",
             "hatsune",
             "aozora",
             ]

In [123]:
stop_words = [ "○－", "○○/", ")○/",
                 "～」", "○－", "○○/", ")○/", "～。", "○/", "://", "com", '）"', ",\u3000", "\u3000-",
                "◎/", "◎◎/", "）⚪", ")、", "^)", "(^", "cp", "ーー",
                '""', "！⚫",]

In [196]:
def normalize_number(text):
    # 連続した数字を0で置換
    replaced_text = re.sub(r'\d+', '0', text)
    return replaced_text

In [215]:
def tokenize_ja(text, lower):
    #text = replaced_text = re.sub(r"\d+", "0", text)
    node = tagger.parseToNode(str(text))
    while node:
        surface = node.surface
        if lower and node.feature.split(',')[0] in ["名詞","形容詞", "動詞", "副詞", "助動詞", "接続詞", "連体詞", "感動詞", "接頭詞"] and surface not in stop_words:
            #分かち書きで取得する品詞を指定
            yield surface
            #lowerで小文字に変換 正規化
        node = node.next

In [198]:
text = "今日はいい天気です。"
tokenize_ja(text, True)

<generator object tokenize_ja at 0x115c2bde0>

In [216]:
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

In [217]:
def frequent_lunch(path):
    f_words_list = []
    f_words = []
    f_words_count = []
    review_list = [] #レビューごとに全単語を収納
    df = pd.read_csv(path)
    for i in df["lunch_review"]:
        txt = tokenize(i, 0, 10000, True) #全単語を収納したリスト
        review_list.append(txt)
        for t in txt:
            f_words_list.append(t)
    counter = Counter(f_words_list)
    for word, count in counter.most_common(20000):
        f_words.append(word)
        f_words_count.append(count)
    return f_words, f_words_count, len(f_words_list), review_list

In [218]:
def frequent_dinner(path):
    f_words_list = []
    f_words = []
    f_words_count = []
    review_list = []
    df = pd.read_csv(path)
    for i in df["dinner_review"]:
        txt = tokenize(i, 0, 10000, True)
        review_list.append(txt)
        for t in txt:
            f_words_list.append(t)
    counter = Counter(f_words_list)
    for word, count in counter.most_common(20000):
        f_words.append(word)
        f_words_count.append(count)
    return f_words, f_words_count, len(f_words_list), review_list

In [219]:
many_words_lunch = []
many_count_lunch = []
word_times_lunch = [] #レビューの全単語数
review_list_lunch = [] #各レビューの全単語
for path in lunch_file:
    many_word, word_count, word_times, review_list = frequent_lunch(path)
    many_words_lunch.append(many_word)
    many_count_lunch.append(word_count)
    word_times_lunch.append(word_times)
    review_list_lunch.append(review_list)

In [220]:
many_words_dinner = []
many_count_dinner = []
word_times_dinner = [] #レビューの全単語数
review_list_dinner = []
for path in dinner_file:
    many_word, word_count, word_times, review_list = frequent_dinner(path)
    many_words_dinner.append(many_word)
    many_count_dinner.append(word_count)
    word_times_dinner.append(word_times)
    review_list_dinner.append(review_list)

In [175]:
len(review_list_lunch[0])

113

In [204]:
for i in range(len(many_words_lunch)):
    a = []
    for j in range(len(many_words_lunch[i])):
        s = 0 #many_words_lunch[i][j]が含まれるレビューの数
        for review in review_list_lunch[i]:
            if many_words_lunch[i][j] in review:
                s += 1
        
        a.append([many_words_lunch[i][j], many_count_lunch[i][j], 100 * (many_count_lunch[i][j] / word_times_lunch[i]), 
                  100 * (s / (len(review_list_lunch[i]) + 1)), many_count_lunch[i][j] / (len(review_list_lunch[i]) + 1)])
    df = pd.DataFrame(a, columns = ["頻出単語", "登場回数", "登場回数/全単語数(" + str(word_times_lunch[i]) + ")%"
                                    , "登場レビュー率 /(" +  str(len(review_list_lunch[i])) + ")本", "平均出現率（出現回数/レビュー数）"])
    df.to_csv(store_name[i] + "_lunch.csv", index=False)

In [256]:
for i in range(len(many_words_lunch)):
    a = []
    for j in range(len(many_words_lunch[i])):
        s = 0 #many_words_lunch[i][j]が含まれるレビューの数
        t = 0
        for review in review_list_lunch[i]:
            if many_words_lunch[i][j] in review:
                s += 1
                t += review.count(many_words_lunch[i][j]) / len(review)
        a.append([many_words_lunch[i][j], many_count_lunch[i][j], int(10000 * (many_count_lunch[i][j] / word_times_lunch[i])) / 100, 
                  int(10000 * (s / (len(review_list_lunch[i]) + 1))) / 100, int(100 * many_count_lunch[i][j] / (len(review_list_lunch[i]) + 1)) / 100,
                  int(10000 * t / len(review_list_lunch[i])) / 100, int(100 * many_count_lunch[i][j] / s) / 100])
    df = pd.DataFrame(a, columns = ["頻出単語", "登場回数", "登場回数/全単語数(" + str(word_times_lunch[i]) + ")%"
                                    , "登場レビュー率 /(" +  str(len(review_list_lunch[i])) + "本)", "レビューあたりの登場回数 /回",
                                    "各レビューにおける登場回数をそのレビューの全単語数で割って平均したもの", "登場したレビューにおける平均登場回数",])
    df.to_csv(store_name[i] + "_lunch.csv", index=False)

In [ ]:
for i in range(len(many_words_dinner)):
    a = []
    for j in range(len(many_words_dinner[i])):
        s = 0 #many_words_dinner[i][j]が含まれるレビューの数
        t = 0 #一つ一つのレビューに含まれるmany_words_dinner[i][j]の数をそのレビューの全単語数で割ったもの
        for review in review_list_dinner[i]:
            if many_words_dinner[i][j] in review:
                s += 1
                t += review.count(many_words_dinner[i][j]) / len(review)
        a.append([many_words_dinner[i][j], many_count_dinner[i][j], int(10000 * (many_count_dinner[i][j] / word_times_dinner[i])) / 100, 
                  int(10000 * (s / (len(review_list_dinner[i]) + 1))) / 100, int(100 * many_count_dinner[i][j] / (len(review_list_dinner[i]) + 1)) / 100,
                  int(10000 * t / len(review_list_dinner[i])) / 100, int(100 * many_count_dinner[i][j] / s) / 100])
    df = pd.DataFrame(a, columns = ["頻出単語", "登場回数", "登場回数/全単語数(" + str(word_times_dinner[i]) + ")%"
                                    , "登場レビュー率 /(" +  str(len(review_list_dinner[i])) + "本)", "レビューあたりの登場回数 /回",
                                    "各レビューにおける登場回数をそのレビューの全単語数で割って平均したもの", "登場したレビューにおける平均登場回数"])
    df.to_csv(store_name[i] + "_dinner.csv", index=False)

In [254]:
df = pd.read_csv("imamura_dinner.csv")

In [255]:
df

,頻出単語,登場回数,登場回数/全単語数(24643)%,登場レビュー率 /(110本),レビューあたりの出現回数 /回,各レビューにおける登場回数をそのレビューの全単語数で割って平均したもの,登場したレビューにおける平均登場回数
0,た,925,3.75,93.69,8.33,3.86,8.894231
1,お,402,1.63,86.48,3.62,2.11,4.187500
2,し,349,1.41,88.28,3.14,1.39,3.561224
3,な,346,1.40,83.78,3.11,1.35,3.720430
4,です,336,1.36,69.36,3.02,1.57,4.363636
5,-,228,0.92,9.00,2.05,0.40,22.800000
6,店,208,0.84,81.08,1.87,1.07,2.311111
7,ます,195,0.79,62.16,1.75,0.89,2.826087
8,さ,188,0.76,66.66,1.69,0.73,2.540541
9,まし,183,0.74,58.55,1.64,0.87,2.815385


In [117]:
review_list_lunch[0][5]

['六本木',
 'ある',
 '有名',
 '寿司',
 'すし',
 '高級',
 'です',
 '実は',
 'ランチ',
 'なら',
 '行け',
 'そう',
 'という',
 'こと',
 '有志',
 '募っ',
 '予約',
 '行く',
 'こと',
 'まし',
 '落ち着い',
 '佇まい',
 '予約',
 'かかわら',
 'ちょっと',
 '入り',
 'づらい',
 '今回',
 'カウンター',
 '真ん中',
 'でし',
 'とても',
 'キレイ',
 '店内',
 'です',
 '軽く',
 '緊張',
 'ます',
 'ランチ',
 '種類',
 '予約',
 'とき',
 'もう',
 '伝え',
 'あり',
 'ます',
 'にぎり',
 'おまかせ',
 'コース',
 '蓋物',
 '付き',
 'にぎり',
 'おまかせ',
 'コース',
 '蓋物',
 '付き',
 'シャンパン',
 'ワイン',
 'メニュー',
 'いっぱい',
 'あり',
 'ます',
 'ランチ',
 'ので',
 '自重',
 '今回',
 'おまかせ',
 'コース',
 '選択',
 'さて',
 'コース',
 '始まり',
 'です',
 'マグロ',
 '赤身',
 'マグロ',
 '！#',
 '本木',
 '寿司',
 'すし',
 'ランチ',
 '東京',
 'マグロ',
 '西麻布',
 'hangover',
 'mealhttps',
 'eat',
 'hangover',
 'work',
 '最初',
 'マグロ',
 '赤身',
 '使っ',
 '寿司',
 'って',
 '食べ',
 'こと',
 'なかっ',
 'です',
 'まろやか',
 'コク',
 '旨味',
 'あり',
 'ます',
 'そして',
 '想像',
 '以上',
 'ふわっと',
 'シャリ',
 '本当に',
 '入れる',
 'ほぐれ',
 'いき',
 'ます',
 'まで',
 '寿司',
 '食べ物',
 '感じ',
 'ます',
 'ほうぼう',
 'ほうぼう',
 '握り',
 '！#',
 'ランチ',
 '本木',
 '寿司',
 'すし',
 '西麻布',
 '東京',
 'ほうぼう',
 'っ